<a href="https://colab.research.google.com/github/annikabrundyn/autodiff/blob/conv/autodiff/tests/convolutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Thu May 13 17:37:31 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import numpy as np
from numba import jit, njit, prange, cuda
import cupy as cp
from cupy import multiply
import math

In [ ]:
!pip install -U cupy
!pip install line_profiler
%load_ext line_profiler

     |████████████████████████████████| 71kB 5.9MB/s 


In [ ]:
# Utils (from Github)

def get_indices(X_shape, HF, WF, stride, pad):
    """
        Returns index matrices in order to transform our input image into a matrix.
        Parameters:
        -X_shape: Input image shape.
        -HF: filter height.
        -WF: filter width.
        -stride: stride value.
        -pad: padding value.
        Returns:
        -i: matrix of index i.
        -j: matrix of index j.
        -d: matrix of index d.
            (Use to mark delimitation for each channel
            during multi-dimensional arrays indexing).
    """
    # get input size
    m, n_C, n_H, n_W = X_shape

    # get output size
    out_h = int((n_H + 2 * pad - HF) / stride) + 1
    out_w = int((n_W + 2 * pad - WF) / stride) + 1

    # ----Compute matrix of index i----

    # Level 1 vector.
    level1 = np.repeat(np.arange(HF), WF)
    # Duplicate for the other channels.
    level1 = np.tile(level1, n_C)
    # Create a vector with an increase by 1 at each level.
    everyLevels = stride * np.repeat(np.arange(out_h), out_w)
    # Create matrix of index i at every levels for each channel.
    i = level1.reshape(-1, 1) + everyLevels.reshape(1, -1)

    # ----Compute matrix of index j----

    # Slide 1 vector.
    slide1 = np.tile(np.arange(WF), HF)
    # Duplicate for the other channels.
    slide1 = np.tile(slide1, n_C)
    # Create a vector with an increase by 1 at each slide.
    everySlides = stride * np.tile(np.arange(out_w), out_h)
    # Create matrix of index j at every slides for each channel.
    j = slide1.reshape(-1, 1) + everySlides.reshape(1, -1)

    # ----Compute matrix of index d----

    # This is to mark delimitation for each channel
    # during multi-dimensional arrays indexing.
    d = np.repeat(np.arange(n_C), HF * WF).reshape(-1, 1)

    return i, j, d


def im2col(X, HF, WF, stride, pad):
    """
        Transforms our input image into a matrix.
        Parameters:
        - X: input image.
        - HF: filter height.
        - WF: filter width.
        - stride: stride value.
        - pad: padding value.
        Returns:
        -cols: output matrix.
    """
    # Padding
    X_padded = np.pad(X, ((0, 0), (0, 0), (pad, pad), (pad, pad)), mode='constant')
    i, j, d = get_indices(X.shape, HF, WF, stride, pad)
    # Multi-dimensional arrays indexing.
    cols = X_padded[:, d, i, j]
    cols = np.concatenate(cols, axis=-1)
    return cols


def col2im(dX_col, X_shape, HF, WF, stride, pad):
    """
        Transform our matrix back to the input image.
        Parameters:
        - dX_col: matrix with error.
        - X_shape: input image shape.
        - HF: filter height.
        - WF: filter width.
        - stride: stride value.
        - pad: padding value.
        Returns:
        -x_padded: input image with error.
    """
    # Get input size
    N, D, H, W = X_shape
    # Add padding if needed.
    H_padded, W_padded = H + 2 * pad, W + 2 * pad
    X_padded = np.zeros((N, D, H_padded, W_padded))

    # Index matrices, necessary to transform our input image into a matrix.
    i, j, d = get_indices(X_shape, HF, WF, stride, pad)
    # Retrieve batch dimension by spliting dX_col N times: (X, Y) => (N, X, Y)
    dX_col_reshaped = np.array(np.hsplit(dX_col, N))
    # Reshape our matrix back to image.
    # slice(None) is used to produce the [::] effect which means "for every elements".
    np.add.at(X_padded, (slice(None), d, i, j), dX_col_reshaped)
    # Remove padding from new image if needed.
    if pad == 0:
        return X_padded
    elif type(pad) is int:
        return X_padded[pad:-pad, pad:-pad, :, :]

In [ ]:
# Convolution, standard implementation

@njit
def convolution(X, W, b, in_ch, out_ch, filter_size, stride, pad):
  """
  Computes convolution of filter over image using element-wise multiplication

  Input:
    - X: input image
    - W: weight matrix
    - b: bias vector
    - in_ch: input channel
    - out_ch: output channel
    - filter_size: num rows/cols of square filter
    - stride: stride value
    - pad: padding value
  Output:
    - output matrix after the convolution
  """
  batches, num_channels, prev_height, prev_width = X.shape

  # Calculate output size.
  height = int((prev_height + 2 * pad - filter_size) / stride) + 1
  width = int((prev_width + 2 * pad - filter_size) / stride) + 1

  out = np.zeros((batches, num_channels, prev_height, prev_width))

  for i in range(batches):  # For each image.

      for c in range(num_channels):  # For each channel.

          for h in range(height):  # Slide the filter vertically.
              h_start = h * stride
              h_end = h_start + filter_size

              for w in range(width):  # Slide the filter horizontally.
                  w_start = w * stride
                  w_end = w_start + filter_size

                  # Element wise multiplication + sum.
                  out[i, c, h, w] = np.sum(X[i, :, h_start:h_end, w_start:w_end]
                                            * W[c, ...]) + b[c]
  return out

@njit(parallel=True)
def convolution_parallel(X, W, b, in_ch, out_ch, filter_size, stride, pad):
  """
  Computes convolution of filter over image using element-wise multiplication
  using parallelization of for-loop.

  Input:
    - X: input image
    - W: weight matrix
    - b: bias vector
    - in_ch: input channel
    - out_ch: output channel
    - filter_size: num rows/cols of square filter
    - stride: stride value
    - pad: padding value
  Output:
    - output matrix after the convolution
  """
  batches, num_channels, prev_height, prev_width = X.shape

  # Calculate output size.
  height = int((prev_height + 2 * pad - filter_size) / stride) + 1
  width = int((prev_width + 2 * pad - filter_size) / stride) + 1

  out = np.zeros((batches, num_channels, prev_height, prev_width))

  for i in range(batches):  # For each image.

      for c in range(num_channels):  # For each channel.

          for h in range(height):  # Slide the filter vertically.
              h_start = h * stride
              h_end = h_start + filter_size

              for w in range(width):  # Slide the filter horizontally.
                  w_start = w * stride
                  w_end = w_start + filter_size

                  # Element wise multiplication + sum.
                  out[i, c, h, w] = np.sum(X[i, :, h_start:h_end, w_start:w_end]
                                            * W[c, ...]) + b[c]
  return out

In [ ]:
# Im2Col Convolution

@njit
def im2col_convolution(X, W, b, in_ch, out_ch, filter_size, stride, pad):

  batches, num_channels, prev_height, prev_width = X.shape

  height = int((prev_height + 2 * pad - filter_size) / stride) + 1
  width = int((prev_width + 2 * pad - filter_size) / stride) + 1

  X_col = im2col(X, filter_size, filter_size, stride, pad)
  w_col = W.reshape((out_ch, -1))
  b_col = b.reshape(-1, 1)

  # Perform matrix multiplication.
  out = w_col @ X_col + b_col

  # Reshape back matrix to image.
  out = np.array(np.hsplit(out, batches)).reshape((batches, out_ch, height, width))

  return out


In [ ]:
# Set input image parameters
out_ch = 5
in_ch = 3
filter_size = 3
stride = 1
pad = 0
nsamples = 10
img_dim = 28

# Set random seed
np.random.seed(42)

# Initialize weights and bias
W = np.random.rand(out_ch, in_ch, filter_size, filter_size)
b = np.random.rand(out_ch)

# Initialize input image with random values
X = np.random.rand(nsamples, in_ch, img_dim, img_dim)

# Regular Convolution

In [ ]:
# Convolution without parallelization
%timeit -n 100 convolution.py_func(X=X, W=W, b=b, in_ch=in_ch, out_ch=out_ch, filter_size=filter_size, stride=stride, pad=pad)

100 loops, best of 5: 139 ms per loop


In [ ]:
# Line profiling
%lprun -u 1e-3 -f convolution.py_func convolution.py_func(X=X, W=W, b=b, in_ch=in_ch, out_ch=out_ch, filter_size=filter_size, stride=stride, pad=pad)

In [ ]:
# Convolution with numba (not using parallelization)
%timeit -n 100 convolution(X=X, W=W, b=b, in_ch=in_ch, out_ch=out_ch, filter_size=filter_size, stride=stride, pad=pad)

100 loops, best of 5: 4.73 ms per loop


In [ ]:
# Convolution with numba (using parallelization)
%timeit -n 100 convolution_parallel(X=X, W=W, b=b, in_ch=in_ch, out_ch=out_ch, filter_size=filter_size, stride=stride, pad=pad)

100 loops, best of 5: 50.1 ms per loop


# Convolution using Im2Col  

In [ ]:
# Matrix multiplication (from Homework 10)

@cuda.jit('(float32[:,:], float32[:,:], float32[:,:])')
def matmul_gpu(A, B, C):
    """
    Perform matrix multiplication of C = A * B
    """
    row, col = cuda.grid(2)
    if row < C.shape[0] and col < C.shape[1]:
        tmp = 0.
        for k in range(A.shape[1]):
            tmp += A[row, k] * B[k, col]
        C[row, col] = tmp


In [ ]:
# Convolution without parallelization
%timeit -n 100 im2col_convolution.py_func(X=X, W=W, b=b, in_ch=in_ch, out_ch=out_ch, filter_size=filter_size, stride=stride, pad=pad)

100 loops, best of 5: 2.27 ms per loop


In [ ]:
%lprun -u 1e-3 -f im2col_convolution.py_func im2col_convolution.py_func(X=X, W=W, b=b, in_ch=in_ch, out_ch=out_ch, filter_size=filter_size, stride=stride, pad=pad)

# Im2Col using Numba

In [ ]:
# CPU Host code: transform input image using Im2Col
X_col = im2col(X=X, HF=filter_size, WF=filter_size, stride=stride, pad=pad)
w_col = W.reshape(out_ch, -1)
b_col = b.reshape(-1, 1)

# Move data to GPU
A_GPU = cuda.to_device(X_col) 
B_GPU = cuda.to_device(w_col)
C_GPU = cuda.to_device(np.zeros((w_col.shape[0],X_col.shape[1]), dtype=np.float32))

# Configure thread blocks for CUDA
threadsperblock = (1, 256)
blockspergrid_x = int(math.ceil(A_GPU.shape[0] / threadsperblock[0]))
blockspergrid_y = int(math.ceil(B_GPU.shape[1] / threadsperblock[1]))
blockspergrid = (blockspergrid_x, blockspergrid_y)

In [ ]:
%%timeit
# Convolution with numba and parallelization
matmul_gpu[blockspergrid, threadsperblock](A_GPU,B_GPU, C_GPU)
cuda.synchronize()

100 loops, best of 5: 2.35 ms per loop


# Im2Col using CuPy

In [ ]:
# CPU Host code: transform input image using Im2Col
X_col = im2col(X=X, HF=filter_size, WF=filter_size, stride=stride, pad=pad)
w_col = W.reshape(out_ch, -1)
b_col = b.reshape(-1, 1)

# Move data to GPU
A_GPU = cp.asarray(X_col) 
B_GPU = cp.asarray(w_col)
C_GPU = cp.asarray(np.zeros((w_col.shape[0],X_col.shape[1]), dtype=np.float32))

# Configure thread blocks for CUDA
threadsperblock = (1, 256)
blockspergrid_x = int(math.ceil(A_GPU.shape[0] / threadsperblock[0]))
blockspergrid_y = int(math.ceil(B_GPU.shape[1] / threadsperblock[1]))
blockspergrid = (blockspergrid_x, blockspergrid_y)

In [ ]:
# Convolution with CuPy and parallelization
%%timeit
C_GPU = cp.matmul(B_GPU,A_GPU)
cuda.synchronize()

The slowest run took 7.36 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 5: 43.2 µs per loop
